In [1]:
!pip install pycaret pandas shap


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Predict pm2.5 next 14 days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:/project-term/pm_2.5/export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # คำนวณค่ากลางจากข้อมูลที่ไม่ใช่ outlier
    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    # แทนที่ outlier และ NaN ด้วยค่าข้างเคียง + ค่ากลาง
    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  # เติมค่าที่ขาดหายด้วย Interpolation
    series = series.fillna(median_value)  # ถ้ายังมี NaN อยู่ให้เติมด้วยค่ากลาง

    return series

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5'
df_cleaned["pm_2_5"] = hybrid_imputation(df_cleaned["pm_2_5"])

# สร้างฟีเจอร์ย้อนหลัง (Lag Features)
for lag in [1, 3, 6, 24]:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# เพิ่มฟีเจอร์เชิงเวลา (Hour และ Day of Week)
df_cleaned["hour"] = df_cleaned["datetime"].dt.hour
df_cleaned["dayofweek"] = df_cleaned["datetime"].dt.dayofweek

# ฟังก์ชันเพิ่ม Fourier Terms สำหรับ Weekly Seasonality
def add_fourier_terms(df, period=24 * 7, n_harmonics=4):
    for k in range(1, n_harmonics + 1):
        df[f"sin_{k}"] = np.sin(2 * np.pi * k * df["hour"] / period)
        df[f"cos_{k}"] = np.cos(2 * np.pi * k * df["hour"] / period)
    return df

# เพิ่ม Fourier Terms ลงใน DataFrame
df_cleaned = add_fourier_terms(df_cleaned)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="pm_2_5", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# ปรับแต่งโมเดล Random Forest Regressor
model = create_model("rf")

final_model = finalize_model(model)
save_model(final_model, "D:/project-term/modelPM/model_PM25")
# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 14 วัน (336 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 337)]  # เปลี่ยนจาก 169 → 337

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

# สร้าง DataFrame สำหรับพยากรณ์
future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    "pm_2_5_lag_1": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_3": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_6": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_24": [latest_pm25] * len(future_dates)
})

# ✅ เพิ่มฟีเจอร์เชิงเวลาให้ future_data
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# ✅ เพิ่ม Fourier Terms สำหรับ Weekly Seasonality
future_data = add_fourier_terms(future_data)

# ✅ โหลดโมเดลที่บันทึกไว้
final_model = load_model("D:/project-term/modelPM/model_PM25")

# ✅ Recursive forecasting (พยากรณ์ 336 ชั่วโมง)
for i in range(len(future_data)):
    pred = predict_model(final_model, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]

    if i + 1 < len(future_data):
        future_data.at[i + 1, "pm_2_5_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "pm_2_5_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "pm_2_5_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "pm_2_5_lag_24"] = predicted_value


future_data.to_csv("D:/project-term/pm_2.5/future_pm_data_14days.csv", index=False)
# ✅ ทำนายค่า PM 2.5
predictions = predict_model(final_model, data=future_data)

# ✅ บันทึกผลลัพธ์เป็นไฟล์ CSV
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_pm25_14days.csv", index=False)

print("✅ บันทึกผลลัพธ์การพยากรณ์ 14 วันสำเร็จ!")


KeyboardInterrupt: 

Predict humidity next 14 days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:/project-term/pm_2.5/export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'humidity' เป็นตัวเลข
df["humidity"] = pd.to_numeric(df["humidity"], errors="coerce")

# ลบคอลัมน์ที่ไม่ต้องการ
for col in ["pm_2_5", "temperature"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# ฟังก์ชัน Hybrid Imputation
def hybrid_imputation(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()
    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")
    series = series.fillna(median_value)
    return series

# ใช้ Hybrid Imputation กับ 'humidity'
df["humidity"] = hybrid_imputation(df["humidity"])

# สร้างฟีเจอร์ย้อนหลังของ humidity
for lag in [1, 3, 6, 24]:
    df[f"humidity_lag_{lag}"] = df["humidity"].shift(lag)

# ลบแถวที่มีค่า NaN หลังจากสร้าง Lag
df.dropna(inplace=True)

# ตั้งค่า PyCaret
exp = setup(df, target="humidity", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# สร้างโมเดล Random Forest Regressor
model_humidity = create_model("rf")

final_model = finalize_model(model_humidity)
save_model(final_model, "D:/project-term/modelPM/model_humidity")
# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 14 วัน (336 ชั่วโมง) ----
last_date = df["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 337)]

# ใช้ค่าล่าสุดของ humidity
latest_humidity = df["humidity"].iloc[-1]

future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity_lag_1": [latest_humidity] * len(future_dates),
    "humidity_lag_3": [latest_humidity] * len(future_dates),
    "humidity_lag_6": [latest_humidity] * len(future_dates),
    "humidity_lag_24": [latest_humidity] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(model_humidity, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "humidity_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "humidity_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "humidity_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "humidity_lag_24"] = predicted_value

# ทำนายค่า Humidity
predictions = predict_model(model_humidity, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

# บันทึกผลลัพธ์
future_data.to_csv("D:/project-term/pm_2.5/future_humidity_data_14days.csv", index=False)
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_humidity_14days.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1070
[LightGBM] [Info] Number of data points in the train set: 18344, number of used features: 7
[LightGBM] [Info] Start training from score 48.890407


,Description,Value
0,Session id,42
1,Target,humidity
2,Target type,Regression
3,Original data shape,"(22930, 6)"
4,Transformed data shape,"(22930, 2)"
5,Transformed train set shape,"(18344, 2)"
6,Transformed test set shape,"(4586, 2)"
7,Numeric features,4
8,Date features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0829,16.8178,4.1009,0.7842,0.0869,0.0663
1,3.0645,16.7679,4.0949,0.7779,0.0871,0.0659
2,2.9555,15.1476,3.8920,0.8054,0.0808,0.0625
3,2.9929,16.1054,4.0132,0.7896,0.0853,0.0638
4,3.0264,15.5513,3.9435,0.7869,0.0830,0.0641
5,3.0170,15.0247,3.8762,0.7943,0.0814,0.0640
6,3.0529,16.5437,4.0674,0.7843,0.0854,0.0649
7,3.0148,15.8721,3.9840,0.7854,0.0842,0.0645
8,2.9745,15.0683,3.8818,0.7838,0.0828,0.0640


Transformation Pipeline and Model Successfully Saved


               datetime  prediction_label
0   2025-02-11 00:00:00         43.201765
1   2025-02-11 01:00:00         40.385086
2   2025-02-11 02:00:00         39.311582
3   2025-02-11 03:00:00         36.402889
4   2025-02-11 04:00:00         37.977754
..                  ...               ...
331 2025-02-24 19:00:00         45.124382
332 2025-02-24 20:00:00         44.939363
333 2025-02-24 21:00:00         45.200301
334 2025-02-24 22:00:00         45.450644
335 2025-02-24 23:00:00         46.931626

[336 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅


Predict temperature next 14 days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:\project-term\pm_2.5\export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'temperature' เป็นตัวเลข
df["temperature"] = pd.to_numeric(df["temperature"], errors="coerce")

# ลบคอลัมน์ 'pm_2_5' และ 'humidity' (หากยังอยู่ใน DataFrame)
for col in ["pm_2_5", "humidity"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชัน Hybrid Imputation
def hybrid_imputation(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  
    series = series.fillna(median_value)  

    return series

# ใช้ Hybrid Imputation สำหรับ 'temperature'
df_cleaned["temperature"] = hybrid_imputation(df_cleaned["temperature"])

# สร้างฟีเจอร์ย้อนหลังของ temperature
for lag in [1, 3, 6, 24]:
    df_cleaned[f"temperature_lag_{lag}"] = df_cleaned["temperature"].shift(lag)

# ลบแถวที่มีค่า NaN หลังจากสร้าง Lag
df_cleaned.dropna(inplace=True)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="temperature", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# สร้างโมเดล Random Forest Regressor
model_temp = create_model("rf")

final_model = finalize_model(model_temp)
save_model(final_model, "D:/project-term/modelPM/model_temperature")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 14 วัน (336 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 337)]

# ใช้ค่าล่าสุดของ temperature
latest_temperature = df_cleaned["temperature"].iloc[-1]

future_data = pd.DataFrame({
    "datetime": future_dates,
    "temperature_lag_1": [latest_temperature] * len(future_dates),
    "temperature_lag_3": [latest_temperature] * len(future_dates),
    "temperature_lag_6": [latest_temperature] * len(future_dates),
    "temperature_lag_24": [latest_temperature] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(model_temp, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "temperature_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "temperature_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "temperature_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "temperature_lag_24"] = predicted_value

# ทำนายค่า Temperature
predictions = predict_model(model_temp, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

future_data.to_csv("D:/project-term/pm_2.5/future_temperature_data_14days.csv", index=False)

# บันทึกผลลัพธ์เป็นไฟล์ CSV โดยเลือกเฉพาะ datetime และ prediction_label
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_temperature_data_14days.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")


,Description,Value
0,Session id,42
1,Target,temperature
2,Target type,Regression
3,Original data shape,"(22930, 6)"
4,Transformed data shape,"(22930, 2)"
5,Transformed train set shape,"(18344, 2)"
6,Transformed test set shape,"(4586, 2)"
7,Numeric features,4
8,Date features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0299,1.9676,1.4027,0.7905,0.0392,0.0298
1,1.0257,1.9016,1.3790,0.7833,0.0387,0.0298
2,1.0186,1.8749,1.3693,0.7958,0.0386,0.0297
3,1.0064,1.8364,1.3551,0.7977,0.0379,0.0291
4,1.0251,1.8561,1.3624,0.7862,0.0383,0.0298
5,1.0100,1.8140,1.3468,0.7984,0.0377,0.0292
6,1.0317,1.8831,1.3723,0.7908,0.0385,0.0300
7,1.0200,1.8340,1.3542,0.7869,0.0379,0.0296
8,1.0155,1.7787,1.3337,0.7875,0.0372,0.0294


Transformation Pipeline and Model Successfully Saved


               datetime  prediction_label
0   2025-02-11 00:00:00         33.955350
1   2025-02-11 01:00:00         34.784517
2   2025-02-11 02:00:00         34.018531
3   2025-02-11 03:00:00         33.487160
4   2025-02-11 04:00:00         34.364235
..                  ...               ...
331 2025-02-24 19:00:00         35.580461
332 2025-02-24 20:00:00         35.874314
333 2025-02-24 21:00:00         36.034567
334 2025-02-24 22:00:00         35.065500
335 2025-02-24 23:00:00         35.077006

[336 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅
